In [1]:
!pip install astroquery

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=f336b9d397c2dab2aa22c295088e74a767fc07e042ee4d41ec7c21d084efc623
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
     |████████████████████████████████| 4.5 MB 5.0 MB/s 
     |████████████████████████████████| 832 kB 44.6 MB/s 
     |████████████████████████████████| 48 kB 3.8 MB/s 
     |████████████████████████████████| 4.0 MB 9.0 MB/s 


In [25]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
from astroquery.jplhorizons import Horizons

In [68]:
# Utility

def convert_name(name) -> str:
  """
  change jpl file asteroids names to legal horizon searchable name
  ie. "2022HM4" -> "2022 HM4"
  Thanks @ Jon
  """
  split = re.split('([A-Z])', name)
  post = "".join(split[1:])
  return " ".join([split[0], post]).strip()

def random_epochs(steps=1.0) -> list:
  """
  generate list of 4 julian dates in steps from 2020.1.1 ~ 2022.5.4
  """
  jd_20200101 = 2458849.5000000
  jd_22220504 = 2459703.4999884
  start = np.random.uniform(jd_20200101, jd_22220504)
  epochs = np.arange(start, start+steps*4, steps)
  return epochs

def get_random_horizon(name_index):
  """
  using astroquery horizon to get ephemerides + orbital elements with random object
  """
  epochs = random_epochs()
  obj = convert_name(np.random.choice(name_index))
  eph = Horizons(id=obj, location='T05', epochs=epochs, id_type='smallbody').ephemerides()
  orb = Horizons(id=obj, location='10', epochs=epochs, id_type='smallbody').elements()
  eph = eph.to_pandas().to_numpy()
  orb = orb['e', 'incl', 'a', 'H'].to_pandas().iloc[:1].to_numpy().flatten()
  return eph, orb

def create_dataset(N, name_index):
  Xs = []
  ys = []
  for n in range(N):
    X_, y_ = get_random_horizon(name_index)
    Xs.append(X_)
    ys.append(y_)
  return np.stack(Xs), np.stack(ys)

In [69]:
# load small body database
sbdb_df = pd.read_csv("sbdb_query_results.csv", index_col='pdes', low_memory=False)
sbdb_df.index = sbdb_df.index.map(lambda x: str(x).replace(" ", ""))
sbdb_df['pha'] = sbdb_df['pha'].apply(lambda x: 1 if x == "Y" else 0)
sbdb_df.dropna(inplace=True)
print(sbdb_df.shape)
sbdb_df.head()

(1099143, 9)


,e,a,i,om,w,ma,H,moid,pha
pdes,,,,,,,,,
1,0.0785,2.766,10.59,80.27,73.64,291.38,3.31,1.58,0
2,0.2300,2.771,34.93,172.92,310.70,272.48,4.11,1.23,0
3,0.2569,2.669,12.99,169.85,247.94,261.30,5.28,1.03,0
4,0.0882,2.361,7.14,103.80,151.09,7.03,3.40,1.14,0
5,0.1901,2.575,5.37,141.57,358.74,160.98,6.98,1.10,0


In [70]:
# data split
train_index, test_index = train_test_split(sbdb_df.index, test_size=0.1, random_state=109)
print(train_index.shape)
print(test_index.shape)

(989228,)
(109915,)


In [ ]:
# # generate 1 observation
# X, y = get_random_horizon(train_index)

# generate dataset
for i in range(40):
  X_train, y_train = create_dataset(1000, train_index)
  np.save(f'X_train_{i}.npy', X_train)
  np.save(f'y_train_{i}.npy', y_train)

for i in range(2):
  X_test, y_test = create_dataset(1000, test_index)
  np.save(f'X_test_{i}.npy', X_train)
  np.save(f'y_test_{i}.npy', y_train)

In [67]:
import glob

id = 0

npy_train_files = sorted(glob.glob(f'X_train*.npy'))
arrs = [np.load(f"{fname}") for fname in npy_train_files]
train_arrs = np.vstack(arrs)
np.save(f"X_train_full_{id}_jack.npy", train_arrs)
files.download(f"X_train_full_{id}_jack.npy")

npy_train_files = sorted(glob.glob(f'y_train*.npy'))
arrs = [np.load(f"{fname}") for fname in npy_train_files]
train_arrs = np.vstack(arrs)
np.save(f"y_train_{id}_jack.npy", train_arrs)
files.download(f"y_train_{id}_jack.npy")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# save
np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)
np.save('X_test.npy', X_test)
np.save('y_test.npy', y_test)

from google.colab import files

# download dataset
files.download('X_train.npy')
files.download('y_train.npy')
files.download('X_test.npy')
files.download('y_test.npy')

In [22]:
# load 
test_arr = np.load('X_train_0.npy')
print(test_arr.shape)
test_arr

(2, 4, 6)


array([[[ 1.55403050e+02,  8.91682000e+00,  2.46914600e+01,
         -7.40771000e+00,  2.17880000e+01,  2.45955103e+06],
        [ 1.55571120e+02,  8.86782000e+00,  2.39892600e+01,
         -7.12562000e+00,  2.17750000e+01,  2.45955203e+06],
        [ 1.55734490e+02,  8.82072000e+00,  2.32772500e+01,
         -6.83765000e+00,  2.17620000e+01,  2.45955303e+06],
        [ 1.55893080e+02,  8.77557000e+00,  2.25559100e+01,
         -6.54398000e+00,  2.17490000e+01,  2.45955403e+06]],

       [[ 2.78047210e+02, -1.28591300e+01,  1.58774100e+01,
          8.95494300e+00,  2.34120000e+01,  2.45894731e+06],
        [ 2.78156590e+02, -1.28004500e+01,  1.52226700e+01,
          8.97170100e+00,  2.34000000e+01,  2.45894831e+06],
        [ 2.78261410e+02, -1.27416700e+01,  1.45609000e+01,
          8.98522000e+00,  2.33880000e+01,  2.45894931e+06],
        [ 2.78361600e+02, -1.26828100e+01,  1.38918200e+01,
          8.99542600e+00,  2.33760000e+01,  2.45895031e+06]]])